# Multiscale Migration Model

This notebook implements our model using `numpy` and `pandas` (with `xlrd`). It has been tested to run on Python 3.6. To start, import the required libraries.

In [1]:
import numpy as np
import pandas as pd

There are a number of datasets used in this model. They can all be found in the `/data` subdirectory.

In [2]:
%ls data

'A&B values for RTS.xlsx'
 Country_List_ISO_3166_Codes_Latitude_Longitude.csv
 Freedom_index.xlsx
 other.csv
 PassportIndex.xlsx
 UN_MigrantStockByOriginAndDestination_2015.xlsx
 wb-codes.csv
 wb-original.csv


The following shortcut functions helps locate these data files easily.

In [3]:
def file_path(name):
    """
    Shortcut function to get the relative path to the directory
    which contains the data.
    """
    return "./data/%s" % name

These functions will be useful later when pulling in the data.

In [4]:
def country_codes():
    """
    Build country rows from their names, ISO codes, and Numeric
    Country Codes.
    """
    return (
        pd.read_csv(
            file_path(
                "Country_List_ISO_3166_Codes_Latitude_Longitude.csv"),
            usecols=[0, 2, 3],
            index_col=1,
            keep_default_na=False))

def freedom_index():
    """
    Read data from the Freedom Index.
    """
    # TODO: Add xlrd to requirements.
    xl = pd.ExcelFile(file_path("Freedom_index.xlsx"))
    return xl.parse(1)

def ab_values():
    """
    Read generated A/B values for each country.
    """
    return pd.read_excel(file_path("A&B values for RTS.xlsx")).T

def passport_index():
    """
    Read data from the Passport Index.
    """
    return pd.read_excel(file_path("PassportIndex.xlsx"))

Modify this depending on how many table rows you want to see.

In [5]:
pd.set_option("display.max_rows", 10)

Now that everything is set up, it is time to start processing the data.

### A/B Values

In [6]:
wb_codes = pd.read_csv(file_path("wb-codes.csv"), index_col=0)
ab = ab_values()
ab.index = [wb_codes[wb_codes.index == x]["ISO3"][0] for x in ab_values().index]
ab

,A,B
BDI,72.993,0.0228
NER,113.170,0.0200
MWI,58.581,0.0226
LBR,144.660,0.0197
MOZ,74.334,0.0267
...,...,...
IRL,20473.000,0.0191
NOR,28548.000,0.0161
USA,12001.000,0.0260
CHE,25711.000,0.0193


### Freedom Index

In [7]:
#[cc[cc['Country'] == x]["Alpha-3 code"] if len(cc[cc['Country'] == x]) > 0 else x for x in fi['Country']]
codes = []
other_codes = pd.read_csv(file_path("other.csv"), index_col=0)
cc = country_codes()
fi = freedom_index()
for country in fi['Country']:
    if len(cc[cc['Country'] == country]):
        codes.append(cc[cc['Country'] == country].index[0])
    elif len(other_codes[other_codes.index == country]):
        codes.append(other_codes[other_codes.index == country]["ISO"][0])
    else:
        print(country)
fi['Country'] = codes
fi

,Country,Total Aggr
0,AFG,24
1,ALB,68
2,DZA,35
3,AND,95
4,AGO,24
...,...,...
190,VEN,30
191,VNM,20
192,YEM,14
193,ZMB,56


### Passport Index

In [8]:
pi = passport_index()
#[cc[cc['Country'] == x]["Alpha-3 code"] if len(cc[cc['Country'] == x]) > 0 else x for x in fi['Country']]
codes = []
other_codes = pd.read_csv(file_path("other.csv"), index_col=0)
for country in pi['Country']:
    if len(cc[cc['Country'] == country]):
        codes.append(cc[cc['Country'] == country].index[0])
    elif len(other_codes[other_codes.index == country]):
        codes.append(other_codes[other_codes.index == country]["ISO"][0])
    else:
        print(country)
pi['Country'] = codes
pi

,Country,Rank (1 = most welcoming)
0,KHM,1
1,COM,1
2,CIV,1
3,GNB,1
4,MDG,1
...,...,...
194,AFG,98
195,PRK,98
196,SOM,98
197,SYR,98


In [9]:
passport_index = pi.set_index("Country")
freedom_index = fi.set_index("Country")
#pd.concat([ab, passport_index, freedom_index])
data = ab.join(passport_index).join(freedom_index).dropna()

In [10]:
political_barriers = 2/(1/data['Total Aggr']/sum(1/data['Total Aggr']) +
 data['Rank (1 = most welcoming)']/sum(data['Rank (1 = most welcoming)']))

In [11]:
political_barriers /= sum(political_barriers)

### UN Migration History

In [12]:
un_pd = pd.read_excel(
        file_path(
            "UN_MigrantStockByOriginAndDestination_2015.xlsx"
        ),
        skiprows=15
    )
un_pd.index = un_pd['Unnamed: 1']

In [13]:
names = un_pd.iloc[:,1]
un_codes = un_pd.iloc[:,3]
names_to_un_codes = {names[i]: un_codes[i] for i in range(len(names))}

In [14]:
un_pd = un_pd.iloc[8:275,8:250]

In [15]:
#un_pd.iloc[:,1:250].columns.values

In [16]:
codes = []
missing = []
for code in [names_to_un_codes[x] for x in un_pd.columns.values]:
    if len(cc[cc['Numeric code'] == code]):
        codes.append(cc[cc['Numeric code'] == code].index[0])
    #elif len(other_codes[other_codes['Code'] == country]):
    #    codes.append(other_codes[other_codes['Code'] == country]["ISO"][0])
    else:
        missing.append(code)
missing

[535, 830, 531, 534, 728, 729]

In [17]:
# We want to drop these
drop = [x for x in un_pd.columns.values if x not in names_to_un_codes.keys() or names_to_un_codes[x] in missing]
print(drop)
un_pd = un_pd.drop(drop, axis=1)
un_pd

['Bonaire, Sint Eustatius and Saba', 'Channel Islands', 'Curaçao', 'Sint Maarten (Dutch part)', 'South Sudan', 'Sudan']


,Afghanistan,Albania,Algeria,American Samoa,Andorra,Angola,Anguilla,Antigua and Barbuda,Argentina,Armenia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela (Bolivarian Republic of),Viet Nam,Wallis and Futuna Islands,Western Sahara,Yemen,Zambia,Zimbabwe
Unnamed: 1,,,,,,,,,,,,,,,,,,,,,
Burundi,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Comoros,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Djibouti,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,265.0,NaN,NaN
Eritrea,NaN,NaN,NaN,NaN,NaN,346.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,241.0,324.0
Ethiopia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,145.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Samoa,NaN,NaN,NaN,1588.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,17.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Tokelau,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Tonga,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
missing = []
for code in [names_to_un_codes[x] for x in un_pd.index]:
    if len(cc[cc['Numeric code'] == code]) == 0:
        #codes.append(cc[cc['Numeric code'] == code].index[0])
    #elif len(other_codes[other_codes['Code'] == country]):
    #    codes.append(other_codes[other_codes['Code'] == country]["ISO"][0])
    #else:
        missing.append(code)
missing

[728,
 911,
 912,
 729,
 913,
 914,
 935,
 5500,
 906,
 920,
 5501,
 922,
 908,
 923,
 924,
 830,
 925,
 926,
 904,
 915,
 535,
 531,
 534,
 916,
 931,
 905,
 909,
 927,
 928,
 954,
 957]

In [19]:
drop = [x for x in un_pd.index if x not in names_to_un_codes.keys() or names_to_un_codes[x] in missing]
print(drop)
un_pd = un_pd.drop(drop, axis=0)
un_pd

['South Sudan', 'Middle Africa', 'Northern Africa', 'Sudan', 'Southern Africa', 'Western Africa', 'Asia', 'Central Asia', 'Eastern Asia', 'South-Eastern Asia', 'Southern Asia', 'Western Asia', 'Europe', 'Eastern Europe', 'Northern Europe', 'Channel Islands', 'Southern Europe', 'Western Europe', 'Latin America and the Caribbean', 'Caribbean', 'Bonaire, Sint Eustatius and Saba', 'Curaçao', 'Sint Maarten (Dutch part)', 'Central America', 'South America', 'Northern America', 'Oceania', 'Australia and New Zealand', 'Melanesia', 'Micronesia', 'Polynesia']


,Afghanistan,Albania,Algeria,American Samoa,Andorra,Angola,Anguilla,Antigua and Barbuda,Argentina,Armenia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela (Bolivarian Republic of),Viet Nam,Wallis and Futuna Islands,Western Sahara,Yemen,Zambia,Zimbabwe
Unnamed: 1,,,,,,,,,,,,,,,,,,,,,
Burundi,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Comoros,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Djibouti,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,265.0,NaN,NaN
Eritrea,NaN,NaN,NaN,NaN,NaN,346.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,241.0,324.0
Ethiopia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,145.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Samoa,NaN,NaN,NaN,1588.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,17.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Tokelau,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Tonga,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
un_pd.index = [cc[cc['Numeric code'] == names_to_un_codes[x]].index[0] for x in un_pd.index]
un_pd.columns = [cc[cc['Numeric code'] == names_to_un_codes[x]].index[0] for x in un_pd.columns.values]

In [21]:
un_pd.fillna(1)

,AFG,ALB,DZA,ASM,AND,AGO,AIA,ATG,ARG,ARM,...,URY,UZB,VUT,VEN,VNM,WLF,ESH,YEM,ZMB,ZWE
BDI,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
COM,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
DJI,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,265.0,1.0,1.0
ERI,1.0,1.0,1.0,1.0,1.0,346.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,241.0,324.0
ETH,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,145.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WSM,1.0,1.0,1.0,1588.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,17.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
TKL,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
TON,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
TUV,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [22]:
un_pd

,AFG,ALB,DZA,ASM,AND,AGO,AIA,ATG,ARG,ARM,...,URY,UZB,VUT,VEN,VNM,WLF,ESH,YEM,ZMB,ZWE
BDI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
COM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
DJI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,265.0,NaN,NaN
ERI,NaN,NaN,NaN,NaN,NaN,346.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,241.0,324.0
ETH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,145.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WSM,NaN,NaN,NaN,1588.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,17.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
TKL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
TON,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
TUV,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
set(ab.index).difference(set(un_pd.index))

{'GAZ', 'ROM', 'SDN', 'SSD', 'TMP', 'ZAR', nan}

In [24]:
un_pd.index

Index(['BDI', 'COM', 'DJI', 'ERI', 'ETH', 'KEN', 'MDG', 'MWI', 'MUS', 'MYT',
       ...
       'PLW', 'ASM', 'COK', 'PYF', 'NIU', 'WSM', 'TKL', 'TON', 'TUV', 'WLF'],
      dtype='object', length=226)

In [25]:
set(ab.index).difference(set(fi['Country']))
# TODO: Replace values here.

{'GAZ', 'ROM', 'TMP', 'ZAR', nan}

In [27]:
ab[ab.index=='SSD']

,A,B
SSD,112.74,0.0305


In [28]:
replace = {
    'GAZ': None, # May need to be changed.
    'ROM': 'ROU',
    'SDN': None, # For now, Sudan
    'SSD': None, # and South Sudan are being dropped.
    'TMP', 'TLS', # This should be fixed.
    'ZAR': 'COD'
}

SyntaxError: invalid syntax (<ipython-input-28-53d233eec98d>, line 6)

In [30]:
[print(x) for x in un_pd.columns]

AFG
ALB
DZA
ASM
AND
AGO
AIA
ATG
ARG
ARM
ABW
AUS
AUT
AZE
BHS
BHR
BGD
BRB
BLR
BEL
BLZ
BEN
BMU
BTN
BOL
BIH
BWA
BRA
VGB
BRN
BGR
BFA
BDI
CPV
KHM
CMR
CAN
CYM
CAF
TCD
CHL
CHN
HKG
MAC
COL
COM
COG
COK
CRI
CIV
HRV
CUB
CYP
CZE
PRK
COD
DNK
DJI
DMA
DOM
ECU
EGY
SLV
GNQ
ERI
EST
ETH
FRO
FLK
FJI
FIN
FRA
GUF
PYF
GAB
GMB
GEO
DEU
GHA
GIB
GRC
GRL
GRD
GLP
GUM
GTM
GIN
GNB
GUY
HTI
VAT
HND
HUN
ISL
IND
IDN
IRN
IRQ
IRL
IMN
ISR
ITA
JAM
JPN
JOR
KAZ
KEN
KIR
KWT
KGZ
LAO
LVA
LBN
LSO
LBR
LBY
LIE
LTU
LUX
MDG
MWI
MYS
MDV
MLI
MLT
MHL
MTQ
MRT
MUS
MYT
MEX
FSM
MCO
MNG
MNE
MSR
MAR
MOZ
MMR
NAM
NRU
NPL
NLD
NCL
NZL
NIC
NER
NGA
NIU
MNP
NOR
OMN
PAK
PLW
PAN
PNG
PRY
PER
PHL
POL
PRT
PRI
QAT
KOR
MDA
REU
ROU
RUS
RWA
SHN
KNA
LCA
SPM
VCT
WSM
SMR
STP
SAU
SEN
SRB
SYC
SLE
SGP
SVK
SVN
SLB
SOM
ZAF
ESP
LKA
PSE
SUR
SWZ
SWE
CHE
SYR
TJK
THA
MKD
TLS
TGO
TKL
TON
TTO
TUN
TUR
TKM
TCA
TUV
UGA
UKR
ARE
GBR
TZA
USA
VIR
URY
UZB
VUT
VEN
VNM
WLF
ESH
YEM
ZMB
ZWE


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [40]:
[print(x[1]['A']) for x in ab.iterrows()]

72.993
113.17
58.581
144.66
74.334
86.596
155.36
158.65
94.449
43.254
82.008
111.86
205.08
232.03
147.99
241.04
251.35
128.56
241.72
164.2
97.818
112.74
135.97
239.67
122.51
109.85
421.3
344.6
283.09
226.36
419.09
171.88
477.96
101.4
358.3
543.41
392.2
610.37
778.25
111.85
315.14
336.77
896.39
546.22
245.3
234.41
555.92
630.15
351.9
642.59
656.7
428.16
675.92
693.58
448.37
396.74
905.74
771.65
315.21
724.29
1500.0
704.69
727.28
688.46
1536.3
785.26
1270.9
1217.8
1236.6
639.84
484.9
1555.9
1329.7
380.92
1016.3
534.01
974.95
937.58
942.96
1325.2
1697.8
1020.4
2046.1
1428.2
944.35
992.76
1977.4
1317.6
634.13
808.09
667.04
1223.4
1509.6
936.84
1566.6
2294.7
31529.9
400.46
571.2
1104.5
1478.9
1028.2
2608.6
1880.6
767.1
1283.3
3678.5
1587.1
372.53
1581.8
2201.4
1656.0
310.31
2389.8
2840.7
594.49
1476.3
3776.7
1159.2
1615.8
4283.3
4105.7
2417.4
6523.6
2375.9
3980.4
1620.9
1866.4
3761.4
7789.3
3400.9
3109.6
5103.0
2410.1
5156.5
8922.2
4695.8
1799.2
3021.5
5502.5
8862.0
6682.2
8388.6
12304.0
15

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [45]:
rows = []
for i in range(len(ab)):
    row = []
    val = ab['A'][i]
    for value in ab['A']:
        row.append(val - value)
    rows.append(row)
np.array(rows)

array([[  0.00000000e+00,  -4.01770000e+01,   1.44120000e+01, ...,
         -1.19280070e+04,  -2.56380070e+04,  -3.46530070e+04],
       [  4.01770000e+01,   0.00000000e+00,   5.45890000e+01, ...,
         -1.18878300e+04,  -2.55978300e+04,  -3.46128300e+04],
       [ -1.44120000e+01,  -5.45890000e+01,   0.00000000e+00, ...,
         -1.19424190e+04,  -2.56524190e+04,  -3.46674190e+04],
       ..., 
       [  1.19280070e+04,   1.18878300e+04,   1.19424190e+04, ...,
          0.00000000e+00,  -1.37100000e+04,  -2.27250000e+04],
       [  2.56380070e+04,   2.55978300e+04,   2.56524190e+04, ...,
          1.37100000e+04,   0.00000000e+00,  -9.01500000e+03],
       [  3.46530070e+04,   3.46128300e+04,   3.46674190e+04, ...,
          2.27250000e+04,   9.01500000e+03,   0.00000000e+00]])

In [43]:
ab['A'][0]

72.992999999999995

In [46]:
pd.DataFrame(rows)

,0,1,2,3,4,5,6,7,8,9,...,152,153,154,155,156,157,158,159,160,161
0,0.000,-40.177,14.412,-71.667,-1.341,-13.603,-82.367,-85.657,-21.456,29.739,...,-12064.007,-20362.007,-20716.007,-14738.007,-25465.007,-20400.007,-28475.007,-11928.007,-25638.007,-34653.007
1,40.177,0.000,54.589,-31.490,38.836,26.574,-42.190,-45.480,18.721,69.916,...,-12023.830,-20321.830,-20675.830,-14697.830,-25424.830,-20359.830,-28434.830,-11887.830,-25597.830,-34612.830
2,-14.412,-54.589,0.000,-86.079,-15.753,-28.015,-96.779,-100.069,-35.868,15.327,...,-12078.419,-20376.419,-20730.419,-14752.419,-25479.419,-20414.419,-28489.419,-11942.419,-25652.419,-34667.419
3,71.667,31.490,86.079,0.000,70.326,58.064,-10.700,-13.990,50.211,101.406,...,-11992.340,-20290.340,-20644.340,-14666.340,-25393.340,-20328.340,-28403.340,-11856.340,-25566.340,-34581.340
4,1.341,-38.836,15.753,-70.326,0.000,-12.262,-81.026,-84.316,-20.115,31.080,...,-12062.666,-20360.666,-20714.666,-14736.666,-25463.666,-20398.666,-28473.666,-11926.666,-25636.666,-34651.666
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,20400.007,20359.830,20414.419,20328.340,20398.666,20386.404,20317.640,20314.350,20378.551,20429.746,...,8336.000,38.000,-316.000,5662.000,-5065.000,0.000,-8075.000,8472.000,-5238.000,-14253.000
158,28475.007,28434.830,28489.419,28403.340,28473.666,28461.404,28392.640,28389.350,28453.551,28504.746,...,16411.000,8113.000,7759.000,13737.000,3010.000,8075.000,0.000,16547.000,2837.000,-6178.000
159,11928.007,11887.830,11942.419,11856.340,11926.666,11914.404,11845.640,11842.350,11906.551,11957.746,...,-136.000,-8434.000,-8788.000,-2810.000,-13537.000,-8472.000,-16547.000,0.000,-13710.000,-22725.000
160,25638.007,25597.830,25652.419,25566.340,25636.666,25624.404,25555.640,25552.350,25616.551,25667.746,...,13574.000,5276.000,4922.000,10900.000,173.000,5238.000,-2837.000,13710.000,0.000,-9015.000


In [50]:
pi['Rank (1 = most welcoming)'][0]


1